In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
sys.path.insert(0, '/Users/tak/Github/QEmbedding/')
import Hybrid_nn
import torch
from torch import nn
import data
import pennylane as qml
import embedding

## 0. Getting Started

Load the MNIST Datasets

In [2]:
feature_reduction = 'PCA8'
classes = [0,1]
X_train, X_test, Y_train, Y_test = data.data_load_and_process('mnist', '2', feature_reduction=feature_reduction, classes=classes)
X1_test, X0_test = [], []
for i in range(len(X_test)):
    if Y_test[i] == 1:
        X1_test.append(X_test[i])
    else:
        X0_test.append(X_test[i])
X1_test, X0_test = torch.tensor(X1_test), torch.tensor(X0_test)

2022-12-13 04:37:24.300366: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-13 04:37:24.301679: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 10. Tune using inter_op_parallelism_threads for best performance.
/opt/anaconda3/envs/QC/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1666646835196/work/torch/csrc/utils/tensor_new.cpp:233.)
  # Remove the CWD from sys.path while we load stuff.


Calculate the Trace Distance Before training the embedding

In [3]:
dev = qml.device('default.qubit', wires=8)

@qml.qnode(dev, interface="torch")
def distance_circuit1(inputs): 
    embedding.QuantumEmbedding1(inputs[0:8])
    return qml.density_matrix(wires=range(8))

class Distance(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.qlayer1_distance = qml.qnn.TorchLayer(distance_circuit1, weight_shapes={})
    
    def forward(self, x1, x0, measure):
        rhos1 = self.qlayer1_distance(x1)
        rhos0 = self.qlayer1_distance(x0)
        rho1 = torch.sum(rhos1, dim=0) / len(x1)
        rho0 = torch.sum(rhos0, dim=0) / len(x0)
        rho_diff = rho1 - rho0

        if measure == "Trace":
            eigvals = torch.linalg.eigvals(rho_diff)
            return 0.5 * torch.real(torch.sum(torch.abs(eigvals)))
        elif measure == "Hilbert-Schmidt":
            return 0.5 * torch.real(torch.trace(rho_diff @ rho_diff))
        

D = Distance()
D_trace = D(X1_test, X0_test, "Trace")
D_HS = D(X1_test, X0_test, "Hilbert-Schmidt")
print(f"Trace Distance before: {D_trace}\n")
print(f"Hilbert Schmidt distance before: {D_HS}")

/opt/anaconda3/envs/QC/lib/python3.7/site-packages/pennylane/qnn/torch.py:328: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1666646835196/work/aten/src/ATen/native/Copy.cpp:250.)
  return self.qnode(**kwargs).type(x.dtype)


Trace Distance before: 0.3561354875564575

Hilbert Schmidt distance before: 0.002418074058368802


## 1. Distances After the Training

Model1 Fidelity

In [7]:
Model1_Fidelity_PATH = []
for i in range(5):
    Model1_Fidelity_PATH.append(f"/Users/tak/Github/QEmbedding/Results/earlystop 10 experiments/experiment{i+1}/Model1 Fidelity/Model1_Fidelity.pt")

Model1_Fidelity_Trace_Distances, Model1_Fidelity_HS_Distances = np.array([]), np.array([])
for path in Model1_Fidelity_PATH:
    TraceModel = Hybrid_nn.get_model("DistanceModel1_Trace")
    HSModel = Hybrid_nn.get_model("DistanceModel1_HS")
    TraceModel.load_state_dict(torch.load(path, map_location=torch.device('cpu')))
    HSModel.load_state_dict(torch.load(path, map_location=torch.device('cpu')))
    with torch.no_grad():
        trace_distance = -1 * TraceModel(X1_test, X0_test)
        HS_distance = -1 * HSModel(X1_test, X0_test)
    Model1_Fidelity_Trace_Distances = np.append(Model1_Fidelity_Trace_Distances, trace_distance)
    Model1_Fidelity_HS_Distances = np.append(Model1_Fidelity_HS_Distances, HS_distance)

print(f"Trace Distance After training with Model1_Fidelity: {Model1_Fidelity_Trace_Distances.mean()} ± {Model1_Fidelity_Trace_Distances.std()}")
print(f"Hilbert Schmidt distance After training with Model1_Fidelity: {Model1_Fidelity_HS_Distances.mean()} ± {Model1_Fidelity_HS_Distances.std()}")


Trace Distance After training with Model1_Fidelity: 0.9245068192481994 ± 0.007407519674209706
Hilbert Schmidt distance After training with Model1_Fidelity: 0.42580925226211547 ± 0.016337088045579866


Model1 HSinner

In [9]:
Model1_HSinner_PATH = []
for i in range(5):
    Model1_HSinner_PATH.append(f"/Users/tak/Github/QEmbedding/Results/earlystop 10 experiments/experiment{i+1}/Model1 HSinner/Model1_HSinner.pt")

Model1_HSinner_Trace_Distances, Model1_HSinner_HS_Distances = np.array([]), np.array([])
for path in Model1_HSinner_PATH:
    TraceModel = Hybrid_nn.get_model("DistanceModel1_Trace")
    HSModel = Hybrid_nn.get_model("DistanceModel1_HS")
    TraceModel.load_state_dict(torch.load(path, map_location=torch.device('cpu')))
    HSModel.load_state_dict(torch.load(path, map_location=torch.device('cpu')))
    with torch.no_grad():
        trace_distance = -1 * TraceModel(X1_test, X0_test)
        HS_distance = -1 * HSModel(X1_test, X0_test)
    Model1_HSinner_Trace_Distances = np.append(Model1_HSinner_Trace_Distances, trace_distance)
    Model1_HSinner_HS_Distances = np.append(Model1_HSinner_HS_Distances, HS_distance)

print(f"Trace Distance After training with Model1_HSinner: {Model1_HSinner_Trace_Distances.mean()} ± {Model1_HSinner_Trace_Distances.std()}")
print(f"Hilbert Schmidt distance After training with Model1_HSinner: {Model1_HSinner_HS_Distances.mean()} ± {Model1_HSinner_HS_Distances.std()}")

Trace Distance After training with Model1_HSinner: 0.9513480186462402 ± 0.005406885768950933
Hilbert Schmidt distance After training with Model1_HSinner: 0.4082879424095154 ± 0.006902721921856312


Model2 Fidelity

In [11]:
Model2_Fidelity_PATH = []
for i in range(5):
    Model2_Fidelity_PATH.append(f"/Users/tak/Github/QEmbedding/Results/earlystop 10 experiments/experiment{i+1}/Model2 Fidelity/Model2_Fidelity.pt")

Model2_Fidelity_Trace_Distances, Model2_Fidelity_HS_Distances = np.array([]), np.array([])
for path in Model2_Fidelity_PATH:
    TraceModel = Hybrid_nn.get_model("DistanceModel2_Trace")
    HSModel = Hybrid_nn.get_model("DistanceModel2_HS")
    TraceModel.load_state_dict(torch.load(path, map_location=torch.device('cpu')))
    HSModel.load_state_dict(torch.load(path, map_location=torch.device('cpu')))
    with torch.no_grad():
        trace_distance = -1 * TraceModel(X1_test, X0_test)
        HS_distance = -1 * HSModel(X1_test, X0_test)
    Model2_Fidelity_Trace_Distances = np.append(Model2_Fidelity_Trace_Distances, trace_distance)
    Model2_Fidelity_HS_Distances = np.append(Model2_Fidelity_HS_Distances, HS_distance)

print(f"Trace Distance After training with Model2_Fidelity: {Model2_Fidelity_Trace_Distances.mean()} ± {Model2_Fidelity_Trace_Distances.std()}")
print(f"Hilbert Schmidt distance After training with Model2_Fidelity: {Model2_Fidelity_HS_Distances.mean()} ± {Model2_Fidelity_HS_Distances.std()}")

Trace Distance After training with Model2_Fidelity: 0.9254536986351013 ± 0.0067062265567590824
Hilbert Schmidt distance After training with Model2_Fidelity: 0.4021798253059387 ± 0.011320657015236626


Model2 HSinner

In [12]:
Model2_HSinner_PATH = []
for i in range(5):
    Model2_HSinner_PATH.append(f"/Users/tak/Github/QEmbedding/Results/earlystop 10 experiments/experiment{i+1}/Model2 HSinner/Model2_HSinner.pt")

Model2_HSinner_Trace_Distances, Model2_HSinner_HS_Distances = np.array([]), np.array([])
for path in Model2_HSinner_PATH:
    TraceModel = Hybrid_nn.get_model("DistanceModel2_Trace")
    HSModel = Hybrid_nn.get_model("DistanceModel2_HS")
    TraceModel.load_state_dict(torch.load(path, map_location=torch.device('cpu')))
    HSModel.load_state_dict(torch.load(path, map_location=torch.device('cpu')))
    with torch.no_grad():
        trace_distance = -1 * TraceModel(X1_test, X0_test)
        HS_distance = -1 * HSModel(X1_test, X0_test)
    Model2_HSinner_Trace_Distances = np.append(Model2_HSinner_Trace_Distances, trace_distance)
    Model2_HSinner_HS_Distances = np.append(Model2_HSinner_HS_Distances, HS_distance)

print(f"Trace Distance After training with Model2_HSinner: {Model2_HSinner_Trace_Distances.mean()} ± {Model2_HSinner_Trace_Distances.std()}")
print(f"Hilbert Schmidt distance After training with Model2_HSinner: {Model2_HSinner_HS_Distances.mean()} ± {Model2_HSinner_HS_Distances.std()}")

Trace Distance After training with Model2_HSinner: 0.9346812963485718 ± 0.023772653504941756
Hilbert Schmidt distance After training with Model2_HSinner: 0.3949353039264679 ± 0.018281192484750082
